<p><img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="400" align="left"><img src="imagenes/logoLCCazul.jpg" width="170" align="right"></p>


# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Aprendizaje en redes neuronales: El algoritmo de *backpropagation*

[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 18 de septiembre de 2017.

En esta libreta vamos a practicar y revisar el principal algoritmo de aprendizaje de los pesos de una  red neuronal hacia adelante. Todos los algoritmos más sofisticados son versiones y modificaciones al algoritmo de *backpropagation* o *B-prop*. Para aplicaciones reales vamos autilizar herramientas poderosas como [Tensorflow](https://www.tensorflow.org) o [Theano](http://www.deeplearning.net/software/theano/), pero es importante conocer y saber desarrollar el algoritmo básico para entender como funciona el aprendizaje en general en redes neuronales.

Vamos a asumir que todos han realizado la libreta anterior donde desarrollaron una estructura básica de una red neuronal, así como el algoritmo de reconocimiento, o *feedforward*. Para una explicacion de la estructura, es necesario que se remitan a la libreta anterior. Sin embarog resumirenos los pasos principales:



Empecemos por inicializar los modulos que vamos a requerir, así como las funciones desarrolladas en otras libretas para el calculo en feedforward de redes neuronales artificiales.

In [1]:
import numpy as np
from feedforward import *

Las funciones ya programadas las dejamos en un archivo adjunto que se llama `feedforward.py` el cual contiene las operaciones que se desarrollaron anteriormente y que agregarlas en esta libreta metería más ruido de lo que ayudaría a la comprensión. El archivo se encuentra adjunto y eres libre de consultarlo y revisarlo. En este caso nos comportamos muy poco formales y cargamos todas las funciones en el espacio de trabajo principal. Esto es porque vamos a hacer de cuenta que las acabamos de programar.

En las libretas de *Jupyter*, cuando quieres consultar la documentación de una función utilizas la combinación de teclas *shift*-*tab*. Si lo haces una vez aparece únicamente el docstring en una ventana emergente. Si utilizas *shift*-*tab* dos veces, aparece la documentación extendida (si hay), y con 3 veces la documentación completa pero en ventana emergente. Con 4 veces la documentación completa aparece en un *frame* abajo y se mantiene mientras tecleas, hasta que se cierre. Este ultimo es especialmente útil para revisar documentación de una función mientras decide uno como utilizarla.

#### Ejercicio 1: Revisa la documentación de las funciones que se listan a continuación, y familiarisate con ellas, ya que las vamos a estar utilizando.

In [ ]:
## Revisa la documentación de cada uno de las soguientes funciones

inicializa_red_neuronal
logistica
extendida
feedforward
perdida
perdida_red